In [1]:
#importing necessary things
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import config #you need to make a config.py file in the folder, already in .gitignore so it shouldn't be pushed to repo

In [2]:
#paths to import the csvs
path1 = 'sephora_df.csv'
path2 = 'ulta_df.csv'
sephora = pd.read_csv(path1)
ulta = pd.read_csv(path2)
# print(sephora.head())
# ulta.head()

In [3]:
#this is where you are connecting to the database and are creating the database
con = psycopg2.connect(dbname='postgres',
      user=config.username, host='',
      password=config.password)

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

cur = con.cursor()
cur.execute('CREATE DATABASE {};'.format('makeup_db'))

In [4]:
#connecting to the database that you just created and starting a session
rds_connection_string = "postgres:tronrunner@localhost:5432/makeup_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
# Confirm tables NOTE: should be empty at first and then when run again after the next few statements the tables should show up
engine.table_names()

['sephora', 'ulta', 'all_makeup']

In [10]:
#putting it all in one big table just in case its easier to use for flask
all_makeup = sephora.merge(ulta, how='outer')
all_makeup.tail()

,Brand,Product,Product Type,Price,Rating,Details,Ingredients,URL,Store
1719,Nioxin,"Scalp Therapy Conditioner, System 3 (Color Tre...",conditioner,51.00,4.5,Strengthen and moisturize hair with Nioxin's t...,"Water, Cetyl Alcohol, Stearamidopropyl Dimethy...",https://www.ulta.com/scalp-therapy-conditioner...,Ulta
1720,Living Proof,Restore Conditioner,conditioner,59.00,4.3,Living Proof Restore Conditioner is a moisture...,"Water/Eau/Aqua, Cetearyl Alcohol, Octafluorope...",https://www.ulta.com/restore-conditioner?produ...,Ulta
1721,Wella,Invigo Brilliance Conditioner For Coarse Hair,conditioner,33.84,4.6,Wella INVIGO Brilliance Vibrant Color Conditio...,"Aqua/Water/Eau, Stearyl Alcohol, Cetyl Alcohol...",https://www.ulta.com/invigo-brilliance-conditi...,Ulta
1722,AG Hair,Colour Care Sterling Silver Toning Conditioner,conditioner,48.00,4.4,Blonde or silver hair needs specific condition...,"Water (Aqua, Eau), Glycerin, Caprylic/Capric T...",https://www.ulta.com/colour-care-sterling-silv...,Ulta
1723,AG Hair,Keratin Repair Restore Daily Strengthening Con...,conditioner,58.00,4.1,"Restore bounce and add a soft, silky look and ...","Water (Aqua, Eau), Cetearyl Alcohol, Caprylic/...",https://www.ulta.com/keratin-repair-restore-da...,Ulta


In [11]:
#adding all makeup to the postgres database
all_makeup.to_sql(name='all_makeup', con=engine, index=False)

In [6]:
#adding sephora to the database
sephora.to_sql(name='sephora', con=engine, index=False)

In [7]:
#adding ulta to the database
ulta.to_sql(name='ulta', con=engine, index=False)